In [7]:
!unzip dataset.zip

Archive:  dataset.zip
  inflating: all_applic.dat          
  inflating: all_meta.dat            
  inflating: readme.md               


In [8]:
import re
import numpy as np
import pandas as pd

from typing import List

# pd.set_option('display.max_colwidth', -1)


def extract_CGN(string: str) -> str:
    codes: List[str] = re.findall("CGN-.*\s*[0-9]+[A-Z]*[0-9]*", string)
    if len(codes) == 0:
        return ""
    code: str = re.findall("[0-9]+\S+", codes[0])[0]
    return code



with open("/content/all_applic.dat") as f:
    all_applic = f.read()
all_applic = all_applic.split("*$*")[3:]

docs = list(map(lambda x: x.split("\n")[2], all_applic))
texts = list(map(lambda x: "\n".join(x.split("\n")[3:]), all_applic))

df = pd.DataFrame(data=list(zip(docs, texts)), columns=["docs",  "text"])
df["text"] = df["text"].str.replace("&gt;", "")
df["text"] = df["text"].str.replace("&lt;", "")
df["docs"] = df["docs"].apply(lambda x: re.findall("[0-9]+-[0-9]+", x)[0])
df["CGN"] = df["text"].apply(extract_CGN) # где то 10-15 не нашлось

In [9]:
df

,docs,text,CGN
0,8-8,Синьцзян-Китайская база экологически чистой эн...,202307050014
1,67-9,Хубэй собирает тендер на строительство гидроэн...,202301310002
2,1-10,Глава I Уведомление о торгах\nМесто проведения...,202305150006
3,53-15,SK проект LOT123F Оборудование и персональные ...,20230307002-N1
4,75-15,Глава I Уведомление о торгах\nМесто проведения...,2021223002
...,...,...,...
1422,22-6,"Глава I Уведомление о торгах\nТерритория, в ко...",202305310011
1423,32-14,"Глава I Уведомление о торгах\nТерритория, в ко...",202305100011
1424,73-15,Глава I Уведомление о торгах\nМесто проведения...,20230103001
1425,9-3,"Глава I Уведомление о торгах\nРайон, в котором...",2023070004


In [4]:
import spacy

In [63]:
materials=set([
'(F-станция) ZC-YJY63-26/351',
'(Сталь)',
'0.5-4000 A-31.5kA',
'Внутренний шкаф',
'Вход в шкаф',
'Высоковольтный электропровод ZRC-YJLY23-26/35kV-3',
'высоковольтное электроснабжение',
'высоковольтный кабель ZC-YJLY23-26/35kV3',
'высоковольтный кабельный кабель C',
'высоковольтный электропровод',
'высоковольтный электропровод D',
'высоковольтный электропровод ZC-YJLV22-26/35kV-3',
'высоковольтный электропровод ZRC-YJLY23-26/35',
'матрицы',
'Вторичное переваривание',
'автомобилей с ПТ-шкафом (1250A)',
'вакуумных переключателей на поверхности KYN61-40.5-2500 A-31.5kA',
'контрольно-измерительных устройств',
'низковольтных кабелей ZC-YJLV23-1,8/3kV-3?300 метров 78000',
'1035kV, ZR-YJY-26/35kV-3',
'автомобилей с ПТ-шкафом (1250A)',
'коммутаторов 4',
'низковольтных шкафов электрического давления: 380',
'1250A',
'13kV, ZR-YJY23-1.8/3kV-3*300mm2m 39200 Ветер-обмен',
'1600A',
'1ZRC-YJY63-26/35-1',
'1ZRC-YJY63-26/35-1?300 метров 5400',
'Высоковольтное электроснабжение',
'Противоторный плавильный плавитель (H) PRW12-12/200A блок 200',
'Электропровод ZRC-YJLY23-26/35 3',
'вакуумные блоки',
'высоковольтных электропроводов ZC-YJLV22-26/35kV-3',
'высоковольтных электропроводов ZRC-YJLY23-26/35',
'низковольтных кабеля (станция C) 4,5 МВт',
'низковольтных электрических кабелей F',
'низковольтных электропроводов ZC-YJY23-1.8/3kV3',
'шкафов для высоковольтных шлюзов в рамках проекта Hydong Kong',
'2000 Am 40',
'2000A 3',
'2000Am5',
'220 кВ',
'220 кВ (станция C)',
'220 кВ (станция D)',
'23 (число)',
'23,2 МВт',
'23kV, ZR-YJY23-1.8/3kV-1*300mm2M 5600 Ветерная модификация',
'2500 A',
'2500A',
'2ZRC-YJLY23-26/35-3',
'(G-станция) ZC-YJY63-26/351',
'Высоковольтное электроснабжение',
'Гидроэлектроэнергия ZRC-YJY23-26/35-3',
'высоковольтных кабеля',
'высоковольтных электропроводов M1100',
'высоковольтных электропроводов ZC-YJV22-26/35kV-3',
'высоковольтных электропроводов ZRC-YJLY23-26/35',
'высоковольтных электропроводов ZRC-YJLY23-26/35kV-3x70 метров 3200',
'место и крышка вакуумного прерывания',
'3150A',
'335kV, ZC-YJLY23-26/35kV-3',
'35 кВ, электрический ток 4000А',
'35kV Убежище с электрическим счетчиком и ключом к подключению',
'35kV, ZR-YJY-26/35kV-3?150mm2160',
'высоковольтных электропроводов ZC-YJV22-26/35kV-3',
'высоковольтных электропроводов ZRC-YJY23-26/35kV-3',
'высоковольтных электропроводов ZRC-YJY23-26/35kV-3',
'место и вокзал KYN61-40.5-1250 A, вакуумные блоки 2',
'место и крышка вакуумного прерывания 2',
'низковольтных кабеля ZRC-YJY23-1.8/3 3?300 метров 17920',
'4,2 МВт',
'4,5 МВт',
'4-го вакуумного прерывателя',
'40.5-4000 A-31.5kA',
'4000A',
'435kV, ZC-YJY23-26/35kV -3',
'4SVG/FC',
'4ZRC-YJY23',
'Внутренний кабель ZRC-YJY23-1.8/3-3',
'Высоковольтное электроснабжение',
'высоковольтных электропроводов ZC-YJV22-26/35kV-3',
'высоковольтных электропроводов ZRC-YJY23-8.7/15kV-3',
'535kV, ZC-YJY23-26/35kV -3',
'5PT шкаф',
'5SVG 1',
'5SVG шкаф SF6',
'5ZR-YJY23-26/35',
'Высоковольтное электроснабжение',
'низковольтных кабелей ZRC-YJY23-1,8/3kV-3',
'630A',
'635kV, ZR-YJY-26/35kV-3',
'6ZR-YJY23-26/35',
'Высоковольтное электроснабжение',
'Стены 3',
'бронзовых линий',
'высоковольтных электропроводов M310',
'высоковольтных электропроводов ZR-YJY23-26/35, 3?300 метров 260 литров',
'735kV, ZR-YJY-26/35kV-3?400 мм2',
'7ZR-YJY23-26/35, 3 ? 300 метров на стенде',
'Высоковольтное электроснабжение',
'Изоляция стенки и установочные приложения с шкафом M90',
'низковольтных кабелей ZRC-YJY23-1.8/3-3x300',
'82000A',
'Изоляционный провод на стене и установка приложений, а также шкаф с шкафом Ми60',
'датчиков',
'низковольтных кабелей Ми-187000',
'KYN61-40,51 и 2-е пищеварение',
'KYN61-40.5-1250 A,',
'KYN61-40.5-1250 A-31.5kA',
'KYN61-40.5-1250 A-31.5kA 1 комплекты для SF6',
'KYN61-40.5-1250 A-31.5kA 2 комплекты для SF6',
'KYN61-40.5-1600A',
'KYN61-40.5-4000A,1',
'KYN61-40.5-630 A-31.5kA',
'KYN61-40.5-630 A-31.5kA 1 комплекты для SF6',
'KYN61-40.5-630 A-31.5kA 2 комплекты для SF6',
'M1050',
'M25',
'M5100',
'M5250',
'SF6',
'SVG',
'SVG возвращается',
'ZC-YJLV',
'ZC-YJLV-26/35kV-3',
'ZC-YJLV23',
'ZC-YJLV23-1.8/3kV-3',
'ZC-YJLV23-26/35kV-300',
'ZC-YJLV23-26/35kV-3',
'ZC-YJLV23-26/35kV-3',
'ZC-YJLY23',
'ZC-YJLY23-26/35kV3',
'ZC-YJLY23-26/35kV3',
'ZC-YJV',
'ZC-YJY23',
'ZC-YJY23-0.6/1kV3',
'ZC-YJY23-26/353',
'ZC-YJY63',
'ZC-YJY63-0.6/1kV1',
'ZC-YJY63-26/351',
'ZR-YJY22-26/35kV-300',
'ZRC-YJLY23',
'ZRC-YJLY23-26/35-3x240',
'ZRC-YJLY23-26/35-3x400',
'ZRC-YJLY23-26/35-3x95',
'ZRC-YJLY23-26/35kV',
'ZRC-YJY23',
'ZRC-YJY23-0.6/1kV',
'ZRC-YJY23-1.8/3kV',
'ZRC-YJY23-26/35-3x400',
'ZRC-YJY23-26/353',
'ZRC-YJY63',
'ZRC-YJY63-0.6/1kV',
'ZRC-YJY63-1.8/3kV',
'ZRC-YJY63-26/351',
'ZRC-YJY63-26/351',
'ZRC-YJY63-26/35kV',
'±65Mvar; нейтральная форма узла: место сопротивления.',
'Вакуумные блоки для 3-й фазы 3 KYN61-40.5-1600 A-31.5kA',
'Вакуумные блоки для 5-го ступенчатого диапазона',
'Вакуумный прерыватель 1-го крыла',
'Вакуумный прерыватель на 1-й площадке',
'Ветряная камера',
'Ветряные коммутаторы с осью',
'серверные драйверы',
'системы управления',
'компоненты измерения битов',
'измерения нагрузки',
'Внешне кассетное устройство 9',
'Внутренний',
'Внутренний кабель ZC-YJV-8.7/15kV-3',
'Внутренний кабель ZRC-YJY23-1.8/3-3',
'Внутренний кабель ZRC-YJY63-0.6/1-1',
'Все установки',
'Вся бронзовая труба с 3 метрами,',
'Гидроэлектроэнергия ZRC-YJY23-26/35-3',
'Главная шкаф KYN61-40.5-4000A, вакуумные блоки',
'Группа',
'Запчасти',
'Защитные маски для защиты от ветровой изоляции FDGWS-40.5/630A',
'Защитный шкаф 3.',
'Изменить, разделить',
'Изоляционный плавитель FGRW-40.5/400A блок 100',
'Изоляционный провод на стене и установка приложений',
'Изоляция GW4-40.5 (DW) / 63050',
'Изоляция GW4-40.5(W)/63050',
'Изоляция GW5-40.5 (DW) / 630 групп 50 защитных масок',
'Изоляция GW5-40.5 (W) / 630 блок 50 защитных масок',
'Источник',
'КЯН61-40.5-1250 A-31.5kA',
'МР',
'Ми-10150',
'Ми-32000',
'Ми-3580',
'Ми-4310',
'Ми-6220',
'Ми-7030',
'Мониторинг газов',
'Наружное устройство',
'Низкий кабель (станция D)',
'Низкий кабель ZC-YJV22-1.8/3kV-3',
'Оптическое давление',
'Перевернуть дорогу',
'Перегрузка: 4000 A',
'Плюс',
'По каждому набору',
'Приложение',
'Проект установки 7,15 МВт ветрогенераторной установки для установки 7,15 МВт ветровой энергии в рамках проекта строительства 50 МВт в южном горном горном районе Порт-Хуси, частота ветрогенератора',
'Путешественники',
'Скриншоты матрицы: 3 уровня TMY100*12.5',
'Содержащиеся в приложении к установке',
'Сталь, винт',
'Стены',
'Схема распределения товаров',
'Установка приложений',
'Устройство 30.',
'Устройство плавления HGRW-40.5KV/200A Группа 1800',
'Устройство по одному',
'Утечка газа SF6 в Интернете для разлива газа SF6 в закрытом устройстве 35kV',
'Электрическая линия и линия хранения энергии KYN61-40.5-1250 A,',
'Электрическая цепь / хранилище в КЯН61-40.5-1250 A-31.5kA 90',
'Электрическая цепь/запас',
'Электронный счет 3 наружных щитов',
'Электропровод 260',
'Электропровод 360',
'автомобилей с ПТ-шкафом (1250A)',
'бронзовые проводники, которые должны быть',
'бронзовых линий',
'вакуумный блок',
'высоковольтный кабель ZC-YJLY23-26/35kV3',
'высоковольтный электропровод D',
'высоковольтный электропровод M8900',
'высоковольтных кабелей',
'высоковольтных кабеля',
'высоковольтных электропроводов',
'высоковольтных электропроводов M1750',
'высоковольтных электропроводов M650',
'высоковольтных электропроводов ZRC-YJLY23-26/35-3x120',
'высоковольтных электропроводов ZRC-YJLY23-26/35-3x500',
'дороги',
'кабеля',
'контрольно-измерительных устройств',
'матрицы (сверху)',
'меди, в том числе',
'муравьев',
'нижнего давления',
'низковольтных шкафов электрического давления',
'перемен',
'резервный источник питания',
'станция',
'трансформатор',
'устройства и линии связи',
'шкаф',
'шкаф KYN61-40.5-630 A-31.5kA 10',
'шкаф с шкафом Ми60',
'электрические линии и резервные проводы KYN61-40.5-1250 A вакуумные переключатели,',
'электрический',
'электроэнергией'])

In [72]:
jobs=[
'строительство и эксплуатация электростанций и их вспомогательных сооружений',
'строительство электростанций',
'строительство дренажных отверстий',
'Дороги',
'трамвайные платформы',
'свалки',
'дорожные знаки',
'дорожные камни',
'защитные куски',
'брошенные земли',
'заполнение, погрузка и разгрузка грунта',
'основы ветровой машины',
'матрицы ветровой машины, опорные столбы и другие фундаменты',
'инженерное строительство',
'коробка для сборки коробок',
'бетонная платформа для коробки',
'емкость',
'железный забор',
'Ветряная база',
'коробка для работы',''
'резервирование, захоронение и т.д. с ветряными машинами, застройкой коробки',
'изготовление и установка базовых наблюдательных пунктов для наблюдения за осаждением',
'прокладки, спирали, сталь, приемка, хранение, перегрузка и установка',
'Противокоррозия между башнями и основами',
'матрасы, спирали и стальные трубы обеспечиваются башнями',
'Оптимизация температурного режима для передних насосов и насосов уровня давления для системы F2APA',
'Оптимизация ключевых клапанов на заводе MX']

In [73]:
materials_tender=[]
for i,row in df.iterrows():
  matlist=list(filter(lambda x: re.search(x,row['text']),materials))
  joblist=list(filter(lambda x: re.search(x,row['text']),jobs))
  materials_tender+=[[row['CGN'],matlist,'EQUIPMENT']]
  materials_tender+=[[row['CGN'],joblist,'JOB']]
result=pd.DataFrame(materials_tender).explode(1)
result.columns=['tenderid','workid','type']

In [ ]:
materials_tender=[]
for i,row in df.iterrows():
  matlist=list(filter(lambda x: re.search(x,row['text']),materials))
  joblist=list(filter(lambda x: re.search(x,row['text']),jobs))
  materials_tender+=[[row['CGN'],matlist,'EQUIPMENT']]
  materials_tender+=[[row['CGN'],joblist,'JOB']]

In [81]:
def extract(x):
   return  [x,
              {
                  "entities":
                      list(filter(lambda x: len(x)>0,list(map(lambda x: (re.search(x,row['text']).span()[0],
                                          re.search(x,row['text']).span()[1],
                                          'EQUIPMENT')
                      if re.search(x,row['text']) else [],materials))+\
                      list(map(lambda x: (re.search(x,row['text']).span()[0],
                                          re.search(x,row['text']).span()[1],
                                          'JOB')
                      if re.search(x,row['text']) else [],jobs))  ))
              }
          ]

In [91]:
lst=df.text.apply(lambda x: extract(x))

In [102]:
lst[0]

['Синьцзян-Китайская база экологически чистой энергии в Синьцзяне, Лоуп, 1 млн. кВт оптико-волнового поля PC\n(Контингент: CGN-202307050014)\nТерритория, в которой находятся тендерные проекты: Синьцзян-Уйгурский автономный район и Тэгу, округ Лоуп\nУсловия торгов\nВ рамках этого проекта на базе экологически чистой энергетики в Синьцзяне в Лоупе 1 млн. кВт (проект тендера)\nНомер: CGN-202307050014, одобренный подразделением по утверждению проектов, финансируется за счет средств, полученных от предприятия, и реализовано, торгующее\nChina Solar Energy Successing Ltd. В рамках этого проекта уже имеются условия для проведения торгов и в настоящее время проводятся открытые торги.\nII. ОБЩИЙ ОБЗОР ПРОЕКТОВ, ОГРАНИЧЕННОСТИ И ПЛАНИРОВАНИЯ ПРОЕКТОВ\nРазмер проекта:\nСиньцзян-цзян, Китай, Новая энергетическая база мощностью 1 000 000 000 000 000 000 000 кигаваттских люминесцентных оптических дисков в районе Лоуп\nМесто планирования расположено в юго-западном пригороде округа Лоуп, который располо

In [75]:
result.to_excel('materials.xlsx')

In [70]:
result.dropna().to_excel('materials.xlsx')

In [ ]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm


def convert(path, dataset):
    nlp = spacy.blank("pt")
    db = DocBin()
    for text, annot in tqdm(dataset):
            doc = nlp.make_doc(text)
            ents = []
            for start, end, label in annot["entities"]:
                span = doc.char_span(start, end, label=label, alignment_mode="contract")
                if span is None:
                    print("Skipping entity")
                else:
                    ents.append(span)
                    print(span)
            doc.ents = ents
            db.add(doc)
    db.to_disk(path)

dataset = lst

n=len(dataset)
half=int(n/2)
convert("train.spacy", dataset[:half])
convert("dev.spacy", dataset[half:])

In [95]:
!python -m spacy init config config.cfg --lang pt --pipeline ner --optimize efficiency --force

2023-12-16 22:28:31.792804: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-16 22:28:31.792870: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-16 22:28:31.794807: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-16 22:28:33.701606: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
⚠ To generate a more effective transformer-based config (GPU-only),
install the spacy-transformers package and re-run this command. The config
generated now does not use transformers.
ℹ Generated config template specific for your use case
- 

In [98]:
!python -m spacy train config.cfg --output /content/ --paths.train /content/train.spacy --paths.dev /content/dev.spacy

2023-12-16 22:33:26.546465: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-16 22:33:26.546548: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-16 22:33:26.676190: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-16 22:33:29.273019: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
ℹ Saving to output directory: /content
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipelin

In [ ]:
nlp = spacy.load("/content/model-best")
doc = nlp(df.text[0])
colors = {"JOB": "linear-gradient(315deg, #f5d020, #f53803)"}
options = {"ents": ["JOB"], "colors": colors}
spacy.displacy.render(doc, style="ent", options=options, jupyter=True)